# Pareto Frontier Analysis

## Overview
Multi-objective optimization for insurance decisions using Pareto frontiers.  We explore trade-offs between ROE, bankruptcy risk, and premium cost, identify knee-point solutions, and compare frontier generation methods.

- **Prerequisites**: [optimization/01_optimization_overview](01_optimization_overview.ipynb)
- **Estimated runtime**: 1-2 minutes
- **Audience**: [Practitioner] / [Developer]

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/optimization'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits.git -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

from ergodic_insurance.pareto_frontier import (
    Objective, ObjectiveType, ParetoFrontier, ParetoPoint,
)
from ergodic_insurance import InsuranceProgram, EnhancedInsuranceLayer

plt.style.use("seaborn-v0_8-darkgrid")

# Reproducibility
SEED = 42
np.random.seed(SEED)

## 1. Bi-Objective: ROE vs. Bankruptcy Risk

Define a simple insurance objective function and generate the Pareto frontier.

In [ ]:
def insurance_objectives(x):
    """x[0] = retention ratio (0.3-1.0), x[1] = premium multiplier (1.0-3.0)."""
    retention, premium_mult = x
    roe = 0.15 * (1 + 0.5 * retention) * (1 - 0.1 * (premium_mult - 1))
    risk = 0.02 * (1 + retention) / (1 + 0.5 * premium_mult)
    cost = 100_000 * premium_mult
    return {"ROE": roe, "bankruptcy_risk": risk, "insurance_cost": cost}

objectives_2d = [
    Objective("ROE", ObjectiveType.MAXIMIZE, weight=0.5),
    Objective("bankruptcy_risk", ObjectiveType.MINIMIZE, weight=0.5),
]

frontier_2d = ParetoFrontier(
    objectives=objectives_2d,
    objective_function=insurance_objectives,
    bounds=[(0.3, 1.0), (1.0, 3.0)],
)
points_2d = frontier_2d.generate_weighted_sum(n_points=50)
print(f"Generated {len(points_2d)} Pareto-optimal points")

In [ ]:
# Plot frontier and highlight knee points
knee_points = frontier_2d.get_knee_points(n_knees=3)

x_vals = [p.objectives["bankruptcy_risk"] for p in points_2d]
y_vals = [p.objectives["ROE"] for p in points_2d]

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(x_vals, y_vals, s=30, alpha=0.6, label="Pareto front")
for i, kp in enumerate(knee_points, 1):
    ax.scatter(kp.objectives["bankruptcy_risk"], kp.objectives["ROE"],
              s=120, marker="D", zorder=5, label=f"Knee {i}")
ax.set_xlabel("Bankruptcy Risk")
ax.set_ylabel("ROE")
ax.set_title("ROE vs. Bankruptcy Risk Pareto Frontier")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

for i, kp in enumerate(knee_points, 1):
    print(f"Knee {i}: ROE={kp.objectives['ROE']:.2%}  "
          f"Risk={kp.objectives['bankruptcy_risk']:.3%}  "
          f"Retention={kp.decision_variables[0]:.0%}  "
          f"PremMult={kp.decision_variables[1]:.2f}x")

## 2. Three-Objective Frontier: ROE vs. Risk vs. Cost

In [ ]:
objectives_3d = [
    Objective("ROE", ObjectiveType.MAXIMIZE, weight=0.4),
    Objective("bankruptcy_risk", ObjectiveType.MINIMIZE, weight=0.3),
    Objective("insurance_cost", ObjectiveType.MINIMIZE, weight=0.3),
]

frontier_3d = ParetoFrontier(
    objectives=objectives_3d,
    objective_function=insurance_objectives,
    bounds=[(0.3, 1.0), (1.0, 3.0)],
)
points_3d = frontier_3d.generate_evolutionary(n_generations=50, population_size=40)
print(f"Generated {len(points_3d)} 3D Pareto-optimal points")

In [ ]:
# Interactive 3D scatter
df_3d = frontier_3d.to_dataframe()

fig = go.Figure(data=[go.Scatter3d(
    x=df_3d["bankruptcy_risk"],
    y=df_3d["insurance_cost"],
    z=df_3d["ROE"],
    mode="markers",
    marker=dict(size=5, color=df_3d["ROE"], colorscale="Viridis", showscale=True),
)])
fig.update_layout(
    title="3D Pareto Frontier",
    scene=dict(xaxis_title="Bankruptcy Risk", yaxis_title="Insurance Cost ($)", zaxis_title="ROE"),
    height=600, template="plotly_white",
)
fig.show()

## 3. Method Comparison: Hypervolume Quality

In [ ]:
method_hv = {}
for label, gen_fn in [
    ("Weighted Sum", lambda f: f.generate_weighted_sum(n_points=30)),
    ("Epsilon-Constraint", lambda f: f.generate_epsilon_constraint(n_points=30)),
    ("Evolutionary", lambda f: f.generate_evolutionary(n_generations=50, population_size=30)),
]:
    f = ParetoFrontier(objectives=objectives_2d,
                       objective_function=insurance_objectives,
                       bounds=[(0.3, 1.0), (1.0, 3.0)])
    pts = gen_fn(f)
    hv = f.calculate_hypervolume()
    method_hv[label] = {"points": len(pts), "hypervolume": hv}

print(f"{'Method':<22s} {'Points':>6s} {'Hypervolume':>12s}")
print("-" * 42)
for m, v in method_hv.items():
    print(f"{m:<22s} {v['points']:>6d} {v['hypervolume']:>12.4f}")

## 4. Sensitivity of the Frontier to Retention Range

In [ ]:
retention_ranges = [
    (0.3, 0.6, "Conservative"),
    (0.5, 0.8, "Moderate"),
    (0.7, 1.0, "Aggressive"),
]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for ax, (lo, hi, label) in zip(axes, retention_ranges):
    f = ParetoFrontier(objectives=objectives_2d,
                       objective_function=insurance_objectives,
                       bounds=[(lo, hi), (1.0, 3.0)])
    pts = f.generate_weighted_sum(n_points=30)
    knees = f.get_knee_points(n_knees=1)

    xv = [p.objectives["bankruptcy_risk"] for p in pts]
    yv = [p.objectives["ROE"] for p in pts]
    ax.plot(xv, yv, "b-", alpha=0.5)
    ax.scatter(xv, yv, c="blue", s=20)
    if knees:
        ax.scatter(knees[0].objectives["bankruptcy_risk"],
                   knees[0].objectives["ROE"],
                   color="red", s=100, marker="D", zorder=5, label="Knee")
    ax.set_xlabel("Bankruptcy Risk")
    ax.set_ylabel("ROE")
    ax.set_title(f"{label} ({lo:.0%}-{hi:.0%})")
    ax.grid(True, alpha=0.3)
    ax.legend()

plt.suptitle("Pareto Frontier Sensitivity to Retention Range", fontweight="bold")
plt.tight_layout()
plt.show()

## Key Takeaways

- The Pareto frontier reveals **fundamental trade-offs** between ROE, risk, and cost that cannot be reduced further.
- **Knee points** offer balanced performance -- the best "bang for the buck" along the frontier.
- Different generation methods (weighted-sum, epsilon-constraint, evolutionary) produce similar hypervolumes; evolutionary methods are most flexible for 3+ objectives.
- Frontier shape is **sensitive to the retention range** allowed; conservative constraints narrow the feasible region.

## Next Steps

- [optimization/04_retention_optimization](04_retention_optimization.ipynb) -- detailed deductible optimization with market cycles
- [optimization/02_sensitivity_analysis](02_sensitivity_analysis.ipynb) -- understand which parameters shift the frontier most